In [31]:
import sys
sys.path.append('CONFIG_files/')

In [23]:
import numpy as np
import pandas as pd
import seaborn as sns
import shutil

from matplotlib import pyplot as plt
from tabulate import tabulate

import os, shutil, time, pdb, random
import scipy.stats as stats 
import scipy

from math import pi
from datetime import datetime
from collections import OrderedDict
import pickle

import torch
from torch.utils.data import TensorDataset, DataLoader

from importlib import reload
from models import *
from utils import *
from runmanager import *
from experiment import *
from plot_utils import *
from preprocessing_utils import *
from analysis_seasonal import * 

from sklearn.metrics import mean_squared_error as mse

import matplotlib
matplotlib.rc_file_defaults()
%matplotlib inline

# import CONFIG_SUSHIWAT_BGMM_23_JUNE_2023 as C
import CONFIG as C

pd.options.display.max_columns = None

np.random.seed(4)

%load_ext autoreload
%autoreload 2

device = C.device

C.REGION, C.RUN_NAME, C.split_by

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


('SUSHIWAT', 'SUSHIWAT_BGMM_21_AUG_2023_EXP_4', 'station')

### Import data

In [24]:
data = DataPreprocessing(train_path=C.TRAIN_PATH, start=C.start, end=C.end, 
                add_yesterday = False, 
                basin_filter = None, 
                split_bias_corrected_only = False, 
                filter_incomplete_years = False, 
                include_non_bc_stations = True, 
                split_by = C.split_by)

### Split data into held out sets for K-fold cross validation

In [25]:
data.split_stations(sort_by_elev = True)

### Prepare data

In [26]:
# C.predictors.append('obs_yesterday')
data.input_data(C.predictors, C.predictand, sort_by_quantile=C.sort_by_quantile)
print(C.sort_by_quantile)

False


## Multi-Run: Train model with different hyperparameters

### Model run

In [27]:
experiment_label = C.RUN_NAME

In [28]:
experiment_label, C.K_FOLD

('SUSHIWAT_BGMM_21_AUG_2023_EXP_4', 10)

In [29]:
load_run = None

In [ ]:
if load_run == None:
    src = 'CONFIG.py'
    dst = f'CONFIG_files/CONFIG_{experiment_label}.py'
    shutil.copy(src, dst)

st_test, predictions, importances = multirun(data, C.predictors, C.params, C.epochs, 
                                split_by = 'station', sequential_samples = False, n_samples=1, load_run = load_run, #None, 
                                experiment_label = experiment_label, feature_attribution=False, 
                                show_loss_plot=False)

,run,epoch,loss,valid_loss,test_loss,decision_loss,epoch duration,run duration,lr,batch_size,likelihood_fn,dropout_rate,random_noise,k,model_arch
0,1,1,1.411865,1.123909,1.020516,1.123909,12.266305,12.310094,0.001,32,bgmm,0,0,0,"(VGLM, [])"
1,1,2,1.238527,1.068133,1.021942,1.068133,12.260561,24.604027,0.001,32,bgmm,0,0,0,"(VGLM, [])"
2,1,3,1.236635,1.057511,1.019307,1.057511,12.283524,36.921548,0.001,32,bgmm,0,0,0,"(VGLM, [])"


In [14]:
# table_of_predictions_ks_test(predictions, C.seasons, columns, sample_cols, add_cols)

In [22]:
table_of_predictions_for_metric(predictions, C.seasons, columns, C.n_samples, sample_cols, add_cols, metric = 'smape', prefix='smape')

Model                          JFM mean    JFM median    AM mean    AM median    JJAS mean    JJAS median    OND mean    OND median
-----------------------------  ----------  ------------  ---------  -----------  -----------  -------------  ----------  ------------
Prec                           0.00        0.00          0.00       0.00         0.00         0.00           0.00        0.00
wrf_prcp                       0.23        0.18          0.38       0.35         0.21         0.18           0.39        0.34
wrf_bc_prcp                    0.23        0.18          0.38       0.35         0.21         0.18           0.39        0.34
precip_norris                  0.23        0.18          0.38       0.35         0.21         0.18           0.39        0.34
VGLM_[]_bgmm_B=128_D=0         0.25        0.21          0.36       0.32         0.20         0.15           0.40        0.30
MLP_[10]_bgmm_B=128_D=0        0.24        0.18          0.29       0.25         0.20         0.16      

In [23]:
table_of_predictions_for_metric(predictions, C.seasons, columns, C.n_samples, sample_cols, add_cols, metric = 'edd', prefix='edd')

Model                          JFM mean    JFM median    AM mean    AM median    JJAS mean    JJAS median    OND mean    OND median
-----------------------------  ----------  ------------  ---------  -----------  -----------  -------------  ----------  ------------
Prec                           0.00        0.00          0.00       0.00         0.00         0.00           0.00        0.00
wrf_prcp                       14.21       10.00         12.79      12.00        32.95        34.00          7.52        5.50
wrf_bc_prcp                    14.21       10.00         12.79      12.00        32.95        34.00          7.52        5.50
precip_norris                  14.21       10.00         12.79      12.00        32.95        34.00          7.52        5.50
VGLM_[]_bgmm_B=128_D=0         5.74        4.60          5.85       4.30         10.32        7.70           5.16        4.30
MLP_[10]_bgmm_B=128_D=0        5.86        4.80          4.77       3.70         10.84        8.80      

In [24]:
table_of_predictions_for_metric(predictions, C.seasons, columns, C.n_samples, sample_cols, add_cols, metric = 'ae', prefix='ae')

Model                          JFM mean    JFM median    AM mean    AM median    JJAS mean    JJAS median    OND mean    OND median
-----------------------------  ----------  ------------  ---------  -----------  -----------  -------------  ----------  ------------
Prec                           0.00        0.00          0.00       0.00         0.00         0.00           0.00        0.00
wrf_prcp                       74.51       55.94         53.95      44.58        293.28       191.28         33.85       19.84
wrf_bc_prcp                    74.51       55.94         53.95      44.58        293.28       191.28         33.85       19.84
precip_norris                  74.51       55.94         53.95      44.58        293.28       191.28         33.85       19.84
VGLM_[]_bgmm_B=128_D=0         84.24       63.04         51.28      37.37        273.26       203.65         44.41       37.07
MLP_[10]_bgmm_B=128_D=0        86.69       62.62         45.33      37.92        276.16       187.20

In [ ]:
a = pd.read_csv('results.csv')

b = a.groupby(['k','run']).agg({'valid_loss': 'min', 
                                 'model_arch': 'first', 
                                 'likelihood_fn': 'first',
                                 'lr':'first',
                                 'batch_size':'first',
                                 'dropout_rate':'first',
                               })

c = b.groupby(['run']).agg({'valid_loss': 'mean', 
                        'model_arch': 'first', 
                        'likelihood_fn': 'first',
                        'lr':'first',
                        'batch_size':'first',
                        'dropout_rate':'first',
                         }
                   ).sort_values('valid_loss').reset_index()

c.groupby(['model_arch',
           'likelihood_fn',
           'lr',
           'batch_size',
           'dropout_rate',
           ]).agg({'valid_loss': 'mean'}
                   ).sort_values('valid_loss').reset_index()